# Tratamento dos Dados

In [1]:
import pandas as pd
from unidecode import unidecode

In [2]:
url = 'https://bi.s3.es.gov.br/covid19/MICRODADOS.zip'
# url = 'MICRODADOS.zip'
df = pd.read_csv(url, sep = ';', encoding = 'cp1252')
df.head()

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido
0,2021-04-23,2021-04-22,2021-04-17,2021-04-23,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não Informado,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado
1,2021-04-23,2021-04-21,2021-04-19,2021-04-23,NaN,NaN,NaN,2021-04-23,NaN,Suspeito,...,Não Informado,Não Informado,Não,Não,Não,Inconclusivo,Não Informado,Não Informado,Não Informado,Não Informado
2,2021-04-23,2021-04-22,2021-04-20,2021-04-23,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não Informado,Ignorado,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado,Não Informado
3,2021-04-23,2021-04-22,2021-04-20,NaN,2021-04-23,NaN,NaN,NaN,NaN,Descartados,...,Não,Não,Não,Não,Não,Não Informado,Negativo,Não Informado,Não Informado,Teste rápido Antígeno
4,2021-04-23,2021-04-22,2021-04-19,2021-04-23,NaN,NaN,NaN,2021-04-23,NaN,Suspeito,...,Não Informado,Não Informado,Não,Sim,Não,Inconclusivo,Não Informado,Não Informado,Não Informado,Não Informado


In [16]:
df

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido,Confirmados,Curas,Obitos
0,2020-02-29,2020-04-14,2020-02-22,2020-02-29,NaN,NaN,NaN,2020-04-18,NaT,Confirmados,...,Não,Não,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,1,1,0
1,2020-03-06,2020-04-16,2020-03-04,2020-10-08,NaN,NaN,NaN,2020-12-13,NaT,Confirmados,...,Não,Não,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,1,1,0
2,2020-03-06,2020-04-14,2020-03-04,2020-03-06,NaN,NaN,NaN,2020-05-08,NaT,Confirmados,...,Não Informado,Não Informado,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,1,1,0
3,2020-03-13,2020-04-14,2020-03-12,2020-03-13,NaN,NaN,NaN,2020-04-15,NaT,Confirmados,...,Não Informado,Não Informado,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,1,1,0
4,2020-03-13,2020-04-14,2020-03-10,2020-03-13,NaN,NaN,NaN,2020-04-14,NaT,Confirmados,...,Não,Não,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425678,2021-04-23,2021-04-23,2021-04-16,NaN,2021-04-23,NaN,NaN,NaN,NaT,Confirmados,...,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno,1,0,0
425679,2021-04-23,2021-04-23,2021-04-13,NaN,2021-04-23,NaN,NaN,2021-04-23,NaT,Confirmados,...,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno,1,0,0
425680,2021-04-23,2021-04-23,2021-04-22,NaN,2021-04-23,NaN,NaN,NaN,NaT,Confirmados,...,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno,1,0,0
425681,2021-04-23,2021-04-23,2021-04-18,2021-04-23,NaN,NaN,NaN,NaN,NaT,Confirmados,...,Não,Não,Positivo,Não Informado,Não Informado,Não Informado,Não Informado,1,0,0


In [17]:
df.query('Evolucao == "Óbito pelo COVID-19"').shape

(9092, 48)

In [4]:
print('Classificacão: {}'.format(df['Classificacao'].unique()))
print('Evolução: {}'.format(df['Evolucao'].unique()))

Classificacão: ['Suspeito' 'Descartados' 'Confirmados']
Evolução: ['-' 'Ignorado' 'Cura' 'Óbito pelo COVID-19' 'Óbito por outras causas']


In [5]:
total_casos = df.query('Classificacao == "Confirmados"').shape[0]
total_obitos = df.query('Evolucao == "Óbito pelo COVID-19"').shape[0]
total_curas = df.query('Evolucao == "Cura"').shape[0]

print('TOTAIS')
print('=================')
print('  - CASOS: {}'.format(total_casos))
print('  - ÓBITOS: {} ({}%)'.format(total_obitos, round((total_obitos * 100) / total_casos, 1)))
print('  - CURA: {} ({}%)'.format(total_curas, round((total_curas * 100) / total_casos, 1)))

TOTAIS
  - CASOS: 425683
  - ÓBITOS: 9092 (2.1%)
  - CURA: 401817 (94.4%)


In [6]:
df['DataNotificacao'] = pd.to_datetime(df['DataNotificacao'])
df['DataObito'] = pd.to_datetime(df['DataObito'])
df.sort_values('DataNotificacao', inplace=True)
df = df.query('Classificacao == "Confirmados"').reset_index(drop=True)

## Padroniza nome de municípios e bairros

In [7]:
df['Municipio'] = df['Municipio'].apply(lambda x: unidecode(str(x)).upper())
df['Bairro'] = df['Bairro'].apply(lambda x: unidecode(str(x)).upper())

## Cálculo de Confirmados, Óbitos e Curados

In [8]:
grupo_base = ['DataNotificacao', 'Municipio', 'Bairro']

datas = df[['DataNotificacao']].drop_duplicates().reset_index(drop=True)
municipios_bairros = df[['Municipio', 'Bairro']].drop_duplicates().reset_index(drop=True)
    
datas['key'] = 0
municipios_bairros['key'] = 0

grupo_base

['DataNotificacao', 'Municipio', 'Bairro']

In [31]:
df['Confirmados'] = 1
df['Curas'] = df['Evolucao'].apply(lambda e: 1 if e == 'Cura' else 0)
df['Obitos'] = df['Evolucao'].apply(lambda e: 1 if e == 'Óbito pelo COVID-19' else 0)

df_confirmados_curas = df[['Municipio', 'Bairro', 'DataNotificacao', 'Confirmados', 'Curas']]\
    .groupby(['Municipio', 'Bairro', 'DataNotificacao'])\
    .sum()\
    .reset_index()

df_obitos = df[['Municipio', 'Bairro', 'DataObito', 'Obitos']]\
    .groupby(['Municipio', 'Bairro', 'DataObito'])\
    .sum()\
    .reset_index()\
    .rename({'DataObito': 'DataNotificacao'}, axis=1)

df_counts = df_confirmados_curas.merge(
    df_obitos,
    on=grupo_base,
    how='outer'
).fillna(0)

In [32]:
df_datas_municipios = pd.merge(datas, municipios_bairros, how='outer')[grupo_base]

df_counts = df_datas_municipios.merge(
    df_counts,
    on=grupo_base,
    how='left'
)

df_counts.fillna(
    {
        'Confirmados': 0,
        'Obitos': 0,
        'Curas': 0,
    },
    inplace=True
)

In [34]:
columns_sum = ['Confirmados', 'Obitos', 'Curas']
df_counts_by_week = df_counts.groupby(['Municipio', 'Bairro', pd.Grouper(key='DataNotificacao', freq='7D')])[columns_sum]\
    .sum()\
    .reset_index()\
    .sort_values('DataNotificacao')

df_counts_by_week['ConfirmadosAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Confirmados']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df_counts_by_week['ObitosAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Obitos']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

df_counts_by_week['CurasAcumulado'] = df_counts_by_week[['Municipio', 'Bairro', 'DataNotificacao', 'Curas']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum()

In [35]:
df_counts_by_week.query('Municipio == "SERRA" & Bairro == "COLINA DE LARANJEIRAS"').tail(6)

,Municipio,Bairro,DataNotificacao,Confirmados,Obitos,Curas,ConfirmadosAcumulado,ObitosAcumulado,CurasAcumulado
251814,SERRA,COLINA DE LARANJEIRAS,2021-03-13,71.0,0.0,71.0,2092.0,18.0,2072.0
251815,SERRA,COLINA DE LARANJEIRAS,2021-03-20,59.0,0.0,59.0,2151.0,18.0,2131.0
251816,SERRA,COLINA DE LARANJEIRAS,2021-03-27,59.0,1.0,59.0,2210.0,19.0,2190.0
251817,SERRA,COLINA DE LARANJEIRAS,2021-04-03,53.0,0.0,45.0,2263.0,19.0,2235.0
251818,SERRA,COLINA DE LARANJEIRAS,2021-04-10,50.0,0.0,31.0,2313.0,19.0,2266.0
251819,SERRA,COLINA DE LARANJEIRAS,2021-04-17,19.0,2.0,0.0,2332.0,21.0,2266.0


In [36]:
df_counts_by_week['Obitos'].sum()

9092.0

In [13]:
# df_counts_by_week.to_csv('microdados_pre-processed.csv', sep=',', index=False, encoding='UTF-8')